In [1]:
import pandas as pd
import string
import numpy as np
import random

In [2]:
df_accepts = pd.read_csv('chefmozaccepts.csv')
df_cuisine = pd.read_csv('chefmozcuisine.csv')
df_hours = pd.read_csv('chefmozhours4.csv')
df_parking = pd.read_csv('chefmozparking.csv')
df_geoplaces = pd.read_csv('geoplaces2.csv', encoding='latin-1')
df_rating = pd.read_csv('rating_final.csv')
df_user_cuisine = pd.read_csv('usercuisine.csv')
df_user_payments = pd.read_csv('userpayment.csv')
df_user_profile = pd.read_csv('userprofile.csv')

#### Clean hours dataframe
The hours dataframe contains duplicates, bad headings, inconsistent opening hours and noisy day values. The next cells will fix these issues. 

In [3]:
#Rename the column names and remove duplicates from hours dataframe
print(df_hours.loc[[1,3,2317],:]) #Hours dataframe before cleaning
df_hours.rename(columns={'days;;;;;;;;': 'days'}, inplace=True) 
df_hours.drop_duplicates(inplace=True)

      placeID                     hours            days;;;;;;;;
1      135111              00:00-23:30;              Sat;;;;;;;
3      135110              08:00-19:00;  Mon;Tue;Wed;Thu;Fri;;;
2317   132030  12:00-15:00;15:00-21:00;               Sat;;;;;;


In [4]:
def clean_hours(text):
    '''
    Rules:
    1. Only one openening time per day
    For instance: 12:00-15:00;15:00-21:00; should be 12:00-21:00 (least opening hour until last closing hour)
    2. Should be one in range 00:00-23:30
    3. If the store is open 24h a day, set to 00:00 - 00:00
    4. Should be a string with format "opening-closing"
    '''
    text = list(filter(None,text.split(";"))) #split on opening hours, filter out empty value
    start, end = "23:30", "00:00"
    if len(text) > 1: #multiple opening times in one row. Filter down to one
        for date in text:
            opening = date.split("-")
            if len(opening) <= 1: continue
            if opening[0] < start:
                start = opening[0]
            if opening[1] > end:
                end = opening[1]
    else:
        start, end = text[0].split("-")[0], text[0].split("-")[1]
    if start == end: #Open 24/7
        start, end = "00:00", "00:00"

    return str(start+"-"+end)

round1 = lambda x: clean_hours(x)
cleaned_hours = pd.DataFrame(df_hours.hours.apply(round1))

#Clean the days, and store it as a list
cleaned_days = pd.DataFrame(df_hours.days.apply(lambda x: list(filter(None,x.split(";")))))
print(cleaned_hours.head())
print(cleaned_days.head())

         hours
0  00:00-23:30
1  00:00-23:30
2  00:00-23:30
3  08:00-19:00
4  00:00-00:00
                        days
0  [Mon, Tue, Wed, Thu, Fri]
1                      [Sat]
2                      [Sun]
3  [Mon, Tue, Wed, Thu, Fri]
4                      [Sat]


In [5]:
"Be sure all values is within range 00:00-23:30"
start =  pd.DataFrame(cleaned_hours.hours.apply(lambda x: x.split("-")[0]))
end =  pd.DataFrame(cleaned_hours.hours.apply(lambda x: x.split("-")[1]))
print(start.loc[start.hours < "00:00"])
print(end.loc[end.hours > "24:00"])
#COMPARE cleaned and originally if needed
#pd.DataFrame({'hours':df_hours.hours, 'cleaned':cleaned_hours.hours})

#Seems okay, set the cleaned columns into the dataframe
df_hours["hours"] = cleaned_hours
df_hours["days"] = cleaned_days

Empty DataFrame
Columns: [hours]
Index: []
Empty DataFrame
Columns: [hours]
Index: []


#### Clean geoplaces
The geoplace dataframe contains unnecessary columns, noisy data and a lot of empty/nan values. The next cells will fix these issues

In [6]:
df_geoplaces.drop(['fax', 'zip', 'url'], axis=1, inplace=True)
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,No_Alcohol_Served,none,informal,completely,low,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,Wine-Beer,only at bar,informal,partially,medium,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,No_Alcohol_Served,none,informal,completely,low,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,No_Alcohol_Served,permitted,informal,completely,medium,familiar,t,closed,none


In [7]:
#Find different ways to mention the same locations 
print("Citites: ", df_geoplaces.city.unique()) 
print("States:", df_geoplaces.state.unique())

Citites:  ['Cuernavaca' 's.l.p.' 'San Luis Potosi' 'victoria ' 'victoria'
 'Cd Victoria' '?' 'san luis potosi' 'Jiutepec' 'cuernavaca' 'slp'
 'Soledad' 'san luis potos' 'san luis potosi ' 'Ciudad Victoria'
 'Cd. Victoria' 's.l.p']
States: ['Morelos' 's.l.p.' 'San Luis Potosi' 'tamaulipas' 'Tamaulipas' 'SLP' '?'
 'san luis potosi' 'morelos' 'slp' 'mexico' 'san luis potos' 'S.L.P.']


In [8]:
'''
1. Set similar words with the same meaning to a single value
2. Capitalize strings related to name of a place
3. Replace missing values with numpy NaN
'''
san_luis_potosi = ['SLP', 's.l.p.', 'slp', 'S.L.P.', 'San Luis Potos', 'san luis potos', 'san luis potosi ', 's.l.p']
df_geoplaces = df_geoplaces.replace(to_replace=san_luis_potosi, value="San Luis Potosi")
df_geoplaces = df_geoplaces.replace(to_replace=['Cd Victoria', 'Cd. Victoria', 'Ciudad Victoria'], value="Victoria")

capitalize = lambda x: string.capwords(x)
df_geoplaces['city'] = df_geoplaces['city'].apply(capitalize)
df_geoplaces['name'] = df_geoplaces['name'].apply(capitalize)
df_geoplaces['address'] = df_geoplaces['address'].apply(capitalize)
df_geoplaces['country'] = df_geoplaces['country'].apply(capitalize)
df_geoplaces['state'] = df_geoplaces['state'].apply(capitalize)

df_geoplaces[['address', 'city', 'state', 'country']] = df_geoplaces[['address', 'city', 'state', 'country']].replace(to_replace=['none', '?'], value=np.nan)
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,alcohol,smoking_area,dress_code,accessibility,price,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,No_Alcohol_Served,none,informal,no_accessibility,medium,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,Puesto De Tacos,Esquina Santos Degollado Y Leon Guzman,San Luis Potosi,San Luis Potosi,Mexico,No_Alcohol_Served,none,informal,completely,low,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón De San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,Wine-Beer,only at bar,informal,partially,medium,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,Little Pizza Emilio Portes Gil,Calle Emilio Portes Gil,Victoria,Tamaulipas,NaN,No_Alcohol_Served,none,informal,completely,low,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,Carnitas_mata,Lic. Emilio Portes Gil,Victoria,Tamaulipas,Mexico,No_Alcohol_Served,permitted,informal,completely,medium,familiar,t,closed,none


In [9]:
print(df_geoplaces.state.unique())
print(df_geoplaces.city.unique())
df_geoplaces.isnull().sum(axis=0) #Lets see what columns contain nan values (should only be state,address,city,country)

['Morelos' 'San Luis Potosi' 'Tamaulipas' nan 'Mexico']
['Cuernavaca' 'San Luis Potosi' 'Victoria' nan 'Jiutepec' 'Soledad']


placeID            0
latitude           0
longitude          0
the_geom_meter     0
name               0
address           27
city              18
state             18
country           28
alcohol            0
smoking_area       0
dress_code         0
accessibility      0
price              0
Rambience          0
franchise          0
area               0
other_services     0
dtype: int64

In [10]:
#clat = np.array(df_geoplaces['latitude'].sub(lat).abs())
#cong = np.array(df_geoplaces['longitude'].sub(long).abs())

def closestCity(x):
    lat, long = x[0],x[1]
    df_geo = df_geoplaces.dropna(subset=['city']).loc[:, ['city', 'latitude', 'longitude']] #ignore nan rows
    df_geo.index = range(len(df_geo))
    locations = np.array(df_geo[["latitude", "longitude"]])
    euc_distance = np.linalg.norm(np.array([[lat,long]]) - locations, axis=1)
    closest_val = df_geo.city.loc[np.argsort(euc_distance)[0]]
    return closest_val

def closestState(x):
    lat, long = x[0],x[1]
    df_geo = df_geoplaces.dropna(subset=['state']).loc[:, ['state', 'latitude', 'longitude']] #ignore nan rows
    df_geo.index = range(len(df_geo))
    locations = np.array(df_geo[["latitude", "longitude"]])
    euc_distance = np.linalg.norm(np.array([[lat,long]]) - locations, axis=1)
    closest_val = df_geo.state.loc[np.argsort(euc_distance)[0]]
    return closest_val

def closestAddress(x):
    lat, long = x[0],x[1]
    df_geo = df_geoplaces.dropna(subset=['address']).loc[:, ['address', 'latitude', 'longitude']] #ignore nan rows
    df_geo.index = range(len(df_geo))
    locations = np.array(df_geo[["latitude", "longitude"]])
    euc_distance = np.linalg.norm(np.array([[lat,long]]) - locations, axis=1)
    closest_val = df_geo.address.loc[np.argsort(euc_distance)[0]]
    return closest_val

In [11]:
'''
1. Fill missing values with appropriate methods
Missing values appear in address, city, state and country (by the definition of the dataset)

'''

df_geoplaces['country'] = "Mexico" #We know this is a dataset for mexican restaurant

#We have the longitude and latitude for all places, so city and state could be derived from that information
#with quite high accuracy of being correct. Use a KNN algorithm to decide which address, state and city 
#is closest to the missing value. The address will not be precise, but as close as we can get (could also set to unknown)
closest_city = lambda x: closestCity(x)
closest_state = lambda x: closestState(x)
closest_address = lambda x: closestAddress(x)
df_geoplaces.city =df_geoplaces.city.fillna(df_geoplaces[df_geoplaces.city.isna()][["latitude", "longitude"]].apply(closest_city, axis=1))
df_geoplaces.state = df_geoplaces.state.fillna(df_geoplaces[df_geoplaces.state.isna()][["latitude", "longitude"]].apply(closest_state, axis=1))
df_geoplaces.address = df_geoplaces.address.fillna(df_geoplaces[df_geoplaces.address.isna()][["latitude", "longitude"]].apply(closest_address, axis=1))

#### Clean user profile dataframe
Contains a lot of missing values. Find appropriate methods to fill them with. In this case, I fill missing values based on the probability of each value in the given column. 

In [12]:
df_user_profile.head() #brief look at the dataframe

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69


In [13]:
missing_columns = ["budget", "activity", "hijos", "marital_status", "transport", "ambience", "smoker", "dress_preference"]
df_user_profile[missing_columns] = df_user_profile[missing_columns].replace(to_replace=["?","none"], value=np.nan)
df_user_profile = df_user_profile.dropna(thresh=13) #If there is 4 missing rows, remove the row

In [14]:
"""
This should be the format
smoker: Nominal, Missing: 3, 2 [false,true]
dress_preference:Nominal, Missing: 5, 4 [informal,formal,no preference,elegant]
ambience: Nominal, Missing: 6, 3 [family,friends,solitary]
marital_status: Nominal, Missing: 4, 3 [single,married,widow]
hijos: Nominal, Missing: 11, 3 [independent,kids,dependent]
activity: Nominal, Missing: 7, 4 [student,professional,unemployed,working-class]
budget: Nominal, Missing: 7, 3 [medium,low,high]
transport: Nominal, Missing: 7, 3 [on foot,public,car owner]
"""

#This is just showing how one could use more specific data to generate a statistical decision for what to fill
# the empty smoker value with. 
def fillSmokers(x):
    df = df_user_profile.dropna(subset=['smoker']).loc[:, ['personality', 'drink_level', 'smoker']]
    prob = len(df[df["smoker"]=="true"])/len(df) #Default probability that a user smokes
    
    #Probability that a person with same drink level also smokes
    drink_level = len(df[(df['drink_level']==x["drink_level"]) & (df["smoker"]=="true")])/len(df)
    if drink_level != 0:
        prob = drink_level
    
    if prob > random.random():
        return "true"
    else:
        return "false"

#All smokers got removed when we deleted rows, so no need to apply this one (just for demo purpose)
#df_user_profile.smoker = df_user_profile.smoker.fillna(fillSmokers)

#For the rest of the values, I will just fill in based on the probability of each value
def fillDressPreference(x):
    df = df_user_profile.dropna(subset=['dress_preference']) #Exclude nan rows
    types = list(df.dress_preference.unique()) #Each type to choose between
    prob = [len(df[df["dress_preference"]==t])/len(df) for t in types] #Probability for each type
    return np.random.choice(types, 1, p=prob)[0] #Random sample based on probability for each value

def fillAmbience(x):
    df = df_user_profile.dropna(subset=['ambience']) 
    types = list(df.ambience.unique()) 
    prob = [len(df[df["ambience"]==t])/len(df) for t in types] 
    return np.random.choice(types, 1, p=prob)[0] 

def fillHijos(x):
    df = df_user_profile.dropna(subset=['hijos']) 
    types = list(df.hijos.unique())
    prob = [len(df[df["hijos"]==t])/len(df) for t in types] 
    return np.random.choice(types, 1, p=prob)[0] 

def fillActivity(x):
    df = df_user_profile.dropna(subset=['activity']) 
    types = list(df.activity.unique())
    prob = [len(df[df["activity"]==t])/len(df) for t in types] 
    return np.random.choice(types, 1, p=prob)[0] 

def fillBudget(x):
    df = df_user_profile.dropna(subset=['budget']) 
    types = list(df.budget.unique())
    prob = [len(df[df["budget"]==t])/len(df) for t in types] 
    return np.random.choice(types, 1, p=prob)[0] 

def fillTransport(x):
    df = df_user_profile.dropna(subset=['transport']) 
    types = list(df.transport.unique())
    prob = [len(df[df["transport"]==t])/len(df) for t in types] 
    return np.random.choice(types, 1, p=prob)[0] 

In [15]:
#Update the nan rows based on appropriate methods
df_user_profile.hijos = df_user_profile.hijos.fillna(df_user_profile[df_user_profile.hijos.isna()].apply(lambda x: fillHijos(x), axis=1))
df_user_profile.dress_preference = df_user_profile.dress_preference.fillna(df_user_profile[df_user_profile.dress_preference.isna()].apply(lambda x: fillDressPreference(x), axis=1))
df_user_profile.ambience = df_user_profile.ambience.fillna(df_user_profile[df_user_profile.ambience.isna()].apply(lambda x: fillAmbience(x), axis=1))
df_user_profile.activity = df_user_profile.activity.fillna(df_user_profile[df_user_profile.activity.isna()].apply(lambda x: fillActivity(x), axis=1))
df_user_profile.budget = df_user_profile.budget.fillna(df_user_profile[df_user_profile.budget.isna()].apply(lambda x: fillBudget(x), axis=1))
df_user_profile.transport = df_user_profile.transport.fillna(df_user_profile[df_user_profile.transport.isna()].apply(lambda x: fillTransport(x), axis=1))

### Part 1
2.1.3. What are the names of different restaurants in the state of ‘tamaulipas’?

2.1.4. How many different customers used public transport for going to the
restaurants?

2.1.5. What is the least popular payment method among customers?

2.1.6. How many (different) restaurants work until 19:00 in the evenings?

2.1.7. Which type of cooking practice (rcuisine) is the most common among
restaurants?

2.1.8. What is the percentage of customers who were born between 1980 and
1990?

2.1.9. What is the percentage of students with a medium budget preferring
walking to the restaurants?

In [16]:
#2.1.3 What are the names of different restaurants in the state of ‘tamaulipas’?
df_geoplaces[df_geoplaces['state']=='Tamaulipas']["name"] #16 originally

3             Little Pizza Emilio Portes Gil
4                              Carnitas_mata
6                          Taqueria El Amigo
8                   Pollo_frito_buenos_aires
19                            Tacos El Guero
27                                Cafe Ambar
41                    Hamburguesas La Perica
43                                   Churchs
72                                      Vips
73                                Palomo Tec
76                        Tacos Correcaminos
83                              Tortas Hawai
86             Carreton De Flautas Y Migadas
89                        Gorditas Dona Tota
94                             Little Cesarz
96                      Tacos De La Estacion
103     Carnitas Mata Calle 16 De Septiembre
106                       Puesto De Gorditas
109    Carnitas Mata Calle Emilio Portes Gil
111       Tacos De Barbacoa Enfrente Del Tec
118                       Gorditas Dona Tota
122                                Tacos Abi
123       

In [17]:
#2.1.4 How many different customers used public transport for going to the restaurants?
len(df_user_profile.loc[df_user_profile["transport"]=="public"]) #82 originally

83

In [18]:
#2.1.5 What is the least popular payment method among customers?
#As we can see from the groupby table, Japan_Credit_Bureau is the least popular with 5 customers
df_accepts = df_accepts.replace(to_replace=['VISA'], value="Visa")
df_accepts.groupby('Rpayment').count() 

,placeID
Rpayment,
American_Express,153
Carte_Blanche,7
Diners_Club,42
Discover,11
Japan_Credit_Bureau,5
MasterCard-Eurocard,194
Visa,255
bank_debit_cards,130
cash,500


In [19]:
#2.1.6 How many (different) restaurants work until 19:00 in the evenings?
len(df_hours[df_hours["hours"].apply(lambda x: x.split("-")[1])=="19:00"].drop_duplicates(subset=["placeID"]))

28

In [20]:
#2.1.7 Which type of cooking practice (rcuisine) is the most common among restaurants?
df_cuisine["Rcuisine"].value_counts().idxmax()

'Mexican'

In [21]:
#2.1.8 What is the percentage of customers who were born between 1980 and 1990?
len(df_user_profile[df_user_profile["birth_year"].apply(lambda x: x <= 1990 and x >= 1980)])/len(df_user_profile)

0.6343283582089553

In [22]:
#2.1.9. What is the percentage of students with a medium budget preferring walking to the restaurants?
walkers = df_user_profile[ (df_user_profile["activity"]=="student") & (df_user_profile["budget"]=="medium") & (df_user_profile["transport"]=="on foot")]
len(walkers)/len(df_user_profile)

0.05970149253731343

### Part 2
2.2.1. What are the names of restaurants that do not have public parking lots?

2.2.2. What are the addresses of restaurants which only accept ‘cash’?

2.2.3. Name the cities where the restaurants cook and serve ‘fast food’?

2.2.4. What is the most common ‘rating’ among customers with family?

2.2.5. What types of ‘rcuisine’ received the highest rank from people with ‘low’
budget?

2.2.6. What is the average of ‘service rating’, received from ‘social drinkers’
about restaurants which just served ‘Wine-Beer’?

2.2.7. How many smokers gave zero ‘service rating’ to the restaurants without an
open area?

2.2.8. Find the correlation between different rating categories
((general) rating, food_rating, service_rating) with the price levels of the
restaurants.

In [23]:
#2.2.1 What are the names of restaurants that do not have public parking lots?
not_public_places = df_parking[df_parking["parking_lot"]!="public"]["placeID"]
df_geoplaces.loc[df_geoplaces["placeID"].isin(not_public_places)]["name"]

0                                        Kiku Cuernavaca
1                                        Puesto De Tacos
2                             El Rincón De San Francisco
3                         Little Pizza Emilio Portes Gil
4                                          Carnitas_mata
5                               Restaurant Los Compadres
6                                      Taqueria El Amigo
7                                              Shi Ro Ie
8                               Pollo_frito_buenos_aires
9                                   La Estrella De Dimas
10                                        Restaurante 75
11                             Abondance Restaurante Bar
12                                  El Angel Restaurante
13                             Restaurante Pueblo Bonito
14                           Mcdonalds Parque Tangamanga
15                        Tortas Y Hamburguesas El Gordo
16                                               Sirlone
17                             

In [24]:
#2.2.2. What are the addresses of restaurants which only accept ‘cash’?
unique_restaurants = df_accepts[df_accepts.duplicated(['placeID'],keep=False)==False]
pd.merge(df_geoplaces,unique_restaurants[unique_restaurants["Rpayment"]=="cash"])["address"]

0             Esquina Santos Degollado Y Leon Guzman
1                            Calle Emilio Portes Gil
2                             Lic. Emilio Portes Gil
3                   Calle Mezquite Fracc Framboyanes
4                                            Tampico
5                  Villa De Pozos 192 Villa De Pozos
6                 Villa De Pozos 4497 Villa De Pozos
7                        Industrias 908 Valle Dorado
8                                   Ricardo B. Anaya
9                                Agustin De Iturbide
10                                  Plaza Del Carmen
11                              Carretera Central Sn
12                       40 Norte Civac 1ra. Seccion
13                           Tangamanga 7 Tangamanga
14                         R.b. Anaya Esq. Florencia
15                                       Anahuac 805
16                           Francisco I. Madero 215
17                                          Cd. Mier
18                                  Ricardo B.

In [25]:
#2.2.3. Name the cities where the restaurants cook and serve ‘fast food’?
fastfood_resraurants = df_cuisine[df_cuisine["Rcuisine"]=="Fast_Food"]["placeID"]
df_geoplaces[df_geoplaces["placeID"].isin(fastfood_resraurants)]["city"].drop_duplicates(keep="first")
#pd.merge(fastfood_resraurants,df_geoplaces)["city"]

8             Victoria
14     San Luis Potosi
64          Cuernavaca
102            Soledad
Name: city, dtype: object

In [26]:
#2.2.4. What is the most common ‘rating’ among customers with family?
family_customers = df_user_profile[df_user_profile["ambience"]=="family"]["userID"]
df_rating[df_rating["userID"].isin(family_customers)]["rating"].value_counts().idxmax()

2

In [27]:
#2.2.5. What types of ‘rcuisine’ received the highest rank from people with ‘low’ budget?
#This will find the rcuisine with highest average rating from the low budget people
lowbudget_profiles = df_user_profile[df_user_profile["budget"]=="low"]["userID"] #profiles with low budget
lowbudget_ratings = df_rating[df_rating["userID"].isin(lowbudget_profiles)] #Ratings done by low budget people
pd.merge(lowbudget_ratings,df_cuisine).groupby(['Rcuisine']).mean()["rating"].idxmax()

'Bakery'

In [28]:
#2.2.6. What is the average of ‘service rating’, received from ‘social drinkers’ about restaurants 
#which just served ‘Wine-Beer’?
social_drinkers = df_user_profile.loc[df_user_profile["drink_level"]=="social drinker"]["userID"]
winebeer_ratings = pd.merge(df_geoplaces.loc[df_geoplaces["alcohol"]=="Wine-Beer"]["placeID"],df_rating)
pd.merge(winebeer_ratings,social_drinkers)["service_rating"].mean()

1.3863636363636365

In [29]:
#2.2.7 How many smokers gave zero ‘service rating’ to the restaurants without an open area?
smokers = df_user_profile.loc[df_user_profile["smoker"]=="true"]["userID"]
openarea_ratings = pd.merge(df_geoplaces.loc[df_geoplaces["area"]!="open"]["placeID"],df_rating)
so_ratings = pd.merge(openarea_ratings,smokers)[["userID","service_rating"]]
len(so_ratings[so_ratings["service_rating"]==0].drop_duplicates())

15

In [30]:
#2.2.8. Find the correlation between different rating categories ((general) rating,
#food_rating, service_rating) with the price levels of the restaurants.
df_geoplaces.price.unique() #medium, low, high
numerical_prices = df_geoplaces.replace(to_replace=['high', 'medium', 'low'], value=[2,1,0]) #set them to numerical
pd.merge(numerical_prices[["price","placeID"]],df_rating).corr().loc["price", ["rating","service_rating","food_rating"]]

rating            0.090745
service_rating    0.118696
food_rating       0.051081
Name: price, dtype: float64